In [1]:
import zipfile

# Unzip the folder
with zipfile.ZipFile('lora_model.zip', 'r') as zip_ref:
    zip_ref.extractall('lora_model')


In [2]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [3]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

In [4]:

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


In [5]:
  if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.11.6: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Unsloth 2024.11.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [6]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Answer the question based on the context provided.", # instruction
        "What is the name of the organization that is working for the development of Chitradurga district?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nAnswer the question based on the context provided.\n\n### Input:\nWhat is the name of the organization that is working for the development of Chitradurga district?\n\n### Response:\nThe organization is not explicitly named in the paragraph.<|end_of_text|>']

In [7]:
!pip install flask pyngrok


In [11]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from transformers import AutoTokenizer, AutoModelForCausalLM

# Set Ngrok API key
ngrok.set_auth_token("2lzFTKGguCaQnvqkBnScBWcsxAP_4srWnM1hngHbUPyDLQcw")  # Replace with your Ngrok API key

# Initialize Flask app
app = Flask(__name__)

# Route for generating response based on the user's question
@app.route('/generate', methods=['POST'])
def generate_response():
    # Parse JSON input from the client request
    data = request.json
    question = data.get("input", "")

    # Define the prompt structure with fixed instruction and dynamic question input
    # alpaca_prompt = "{}\n{}\n{}"
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                "Answer the question based on the context provided.",  # instruction
                question,  # input from client
                ""  # output placeholder for model generation
            )
        ],
        return_tensors="pt"
    ).to("cuda")

    # Generate model response
    outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
    response_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # Return the output as JSON
    print(response_text)
    return jsonify({"output": response_text})


if __name__ == "__main__":
    # Start an ngrok tunnel to the Flask server
    public_url = ngrok.connect(5000).public_url
    print(f"Ngrok tunnel URL: {public_url}")

    # Run the Flask server
    app.run(port=5000)


Ngrok tunnel URL: https://98e3-35-197-1-120.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [13/Nov/2024 07:29:59] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/Nov/2024 07:30:38] "POST /generate HTTP/1.1" 200 -


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the question based on the context provided.

### Input:
hii how are you

### Response:
I am fine, thank you.


INFO:werkzeug:127.0.0.1 - - [13/Nov/2024 07:32:10] "POST /generate HTTP/1.1" 200 -


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the question based on the context provided.

### Input:
ok

### Response:
What is the name of the tree that is being discussed?


INFO:werkzeug:127.0.0.1 - - [13/Nov/2024 07:33:40] "POST /generate HTTP/1.1" 200 -


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the question based on the context provided.

### Input:
hii

### Response:
What is the name of the tree that is being discussed and what is its purpose?


INFO:werkzeug:127.0.0.1 - - [13/Nov/2024 07:33:50] "POST /generate HTTP/1.1" 200 -


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the question based on the context provided.

### Input:
ok

### Response:
What is the name of the village where the Dalimbiya Toto is located?


In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from transformers import AutoTokenizer, AutoModelForCausalLM

# Set Ngrok API key
ngrok.set_auth_token("2lzFTKGguCaQnvqkBnScBWcsxAP_4srWnM1hngHbUPyDLQcw")  # Replace with your Ngrok API key

# Initialize Flask app
app = Flask(__name__)

# Route for generating response based on the user's question
@app.route('/generate', methods=['POST'])
def generate_response():
    # Parse JSON input from the client request
    data = request.json
    question = data.get("input", "")

    # Define the prompt structure with fixed instruction and dynamic question input
    # alpaca_prompt = "{}\n{}\n{}"
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                "Answer the question based on the context provided.",  # instruction
                question,  # input from client
                ""  # output placeholder for model generation
            )
        ],
        return_tensors="pt"
    ).to("cuda")

    # Generate model response
    outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
    response_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    response_text_cleaned = response_text.split("###")[-1].strip()
    response_text_cleaned1 = response_text.split(":")[-1].strip()




    # Return the output as JSON
    print(response_text_cleaned1)
    return jsonify({"output": response_text_cleaned1})


if __name__ == "__main__":
    # Start an ngrok tunnel to the Flask server
    public_url = ngrok.connect(5000).public_url
    print(f"Ngrok tunnel URL: {public_url}")

    # Run the Flask server
    app.run(port=5000)

Ngrok tunnel URL: https://e39e-35-197-1-120.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [13/Nov/2024 07:49:26] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/Nov/2024 07:50:01] "POST /generate HTTP/1.1" 200 -


The organization is not explicitly named but is described as a group of people working for the development of Chitradurga district.
